In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

Using TensorFlow backend.


In [27]:
def load_file():
    '''loads the datafile and stores label,text attributes to an SMS_datat object'''
    file_names=['datasets/smsspamcollection/SMSSpamCollection']
    file=0
    max_len=0
    dataset=[]
    a = []
    with open(file_names[file]) as f:
        for line in f:
            words=line.split()
            words=text_processing(' '.join(words[1:]))
            if max_len < len(words):
                max_len = len(words)
                a = words
    print(max_len, a)
load_file()

78 ['indians', 'r', 'poor', 'india', 'poor', 'country', 'says', 'one', 'swiss', 'bank', 'directors', 'says', 'lt', 'gt', 'lac', 'crore', 'indian', 'money', 'deposited', 'swiss', 'banks', 'used', 'taxless', 'budget', 'lt', 'gt', 'yrs', 'give', 'lt', 'gt', 'crore', 'jobs', 'indians', 'village', 'delhi', '4', 'lane', 'roads', 'forever', 'free', 'power', 'suply', 'lt', 'gt', 'social', 'projects', 'every', 'citizen', 'get', 'monthly', 'lt', 'gt', 'lt', 'gt', 'yrs', 'need', 'world', 'bank', 'amp', 'imf', 'loan', 'think', 'money', 'blocked', 'rich', 'politicians', 'full', 'rights', 'corrupt', 'politicians', 'itna', 'forward', 'karo', 'ki', 'pura', 'india', 'padhe', 'g']


In [8]:
import math
import random
import sys
import gensim
import warnings
import re
from itertools import chain
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import wordnet
from nltk.corpus import stopwords

def main():
	
	dataset=load_file() 
	dataset=get_vectors(dataset)

	k=dataset[0]
	print(k.label,k.words,len(k.vectors),len(k.vectors[0]))


def get_vectors(dataset):

	'''the sentences obtained from dataset are sent into word2vec 
	to obtain word wise vectors for each sentence'''
	
	total_sentences=len(dataset)
	sentences=[temp.words for temp in dataset]
	
	model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, limit=600000)
	c=0
	k=0
	flag = 1
	for sentence in range(total_sentences):
		for word in dataset[sentence].words:
			k+=1
			synonyms = wordnet.synsets(word)
			wordnet_synonyms=list(set(chain.from_iterable([word.lemma_names() for word in synonyms])))
			
			if word in model.vocab:
				'''if word exists in word2vec use vector directly'''
				dataset[sentence].vectors.append(model[word])

			elif len(wordnet_synonyms)!=0:
				'''if word does not exist in word2vec, find synonym using wordnet'''
				
				for i in wordnet_synonyms:
					if i in model.vocab:
						#use vector of similar word from wordnet
						dataset[sentence].vectors.append(model[i])
						break
			else:
				'''need to use ConceptNet'''
				c+=1
				# print(word)
				dataset[sentence].vectors.append(model[random.choice(model.index2entity)])
				# dataset[sentence].vectors.append(model[word])

	#print('\nun-identified words:',c,'\ntotal words:',k)
	return dataset


def load_file():
	'''loads the datafile and stores label,text attributes to an SMS_datat object'''
	file_names=['datasets/smsspamcollection/SMSSpamCollection']
	file=0

	dataset=[]

	with open(file_names[file]) as f:
		for line in f:
			words=line.split()

			temp=SMS_data()
			temp.label=words[0] #accessing the label of text (spam or ham)
			temp.words=text_processing(' '.join(words[1:])) #obtaining the text as list of words
			dataset.append(temp)
		print(len(dataset))
	return dataset[:2274]

def text_processing(text):

	words=re.sub('[^A-Za-z0-9]+', ' ', text) #removes special characters
	#words=re.split('(\d+)',words)
	stop = set(stopwords.words('english'))
	words=[i for i in words.lower().split() if i not in stop] #stop word removal

	return words

class SMS_data:
	def __init__(self):
		self.label=None
		self.words=None
		self.vectors=[]

if __name__ == '__main__':
	main()

5574
ham ['go', 'jurong', 'point', 'crazy', 'available', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amore', 'wat'] 16 300


In [2]:
import numpy as np

In [3]:
from keras.models import Model
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten, Input, LSTM
import numpy as np

In [4]:
import math
import random
import sys
import gensim
import warnings
import re
from itertools import chain
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import wordnet
from nltk.corpus import stopwords

In [5]:
words = ['go', 'jurong', 'point', 'crazy', 'available', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amore', 'wat']

In [7]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, limit=600000)

In [35]:
input = []
for w in words :
    if w in model.vocab  :
        input.append(model[w])
    else:
        input.append(model[random.choice(model.index2entity)])
print(input[0])

[-2.63671875e-02  6.83593750e-02 -3.11279297e-02  2.19726562e-01
  3.41796875e-03 -9.03320312e-03  1.07910156e-01 -1.74804688e-01
  7.71484375e-02  3.83377075e-04 -1.02539062e-01 -1.73339844e-02
 -3.08837891e-02  5.76171875e-02 -1.09863281e-01  6.10351562e-02
  2.48046875e-01  5.46264648e-03  3.49121094e-02  7.65991211e-03
 -1.07910156e-01  2.16796875e-01  1.26953125e-01  1.46484375e-01
  1.55273438e-01  4.46777344e-02  7.51953125e-02 -1.45507812e-01
 -7.71484375e-02 -8.54492188e-02 -1.15966797e-02  7.95898438e-02
 -1.94335938e-01 -2.57812500e-01 -9.86328125e-02 -1.38671875e-01
 -4.13894653e-04 -9.03320312e-02  7.66601562e-02  1.33789062e-01
  5.17578125e-02 -4.80957031e-02  1.88476562e-01 -3.44238281e-02
 -7.95898438e-02  3.54003906e-02 -1.02539062e-01 -1.33789062e-01
 -1.06445312e-01  7.42187500e-02 -2.46582031e-02  1.99218750e-01
  1.54296875e-01 -1.22558594e-01  4.71191406e-02 -3.00781250e-01
 -7.37304688e-02 -6.44531250e-02  9.08203125e-02  6.68945312e-02
 -3.68652344e-02 -1.48437

In [36]:
model = Sequential()

In [37]:
model.add(Conv1D(32,4,activation='relu',input_shape=(1200,1)))
model.add(LSTM(100, activation='sigmoid', recurrent_activation='hard_sigmoid', use_bias=True, dropout=0.1, recurrent_dropout=0.0))
model.add(Dense(1, activation='softmax'))

In [38]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1197, 32)          160       
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 53,461
Trainable params: 53,461
Non-trainable params: 0
_________________________________________________________________


In [49]:
input

[array([-2.63671875e-02,  6.83593750e-02, -3.11279297e-02,  2.19726562e-01,
         3.41796875e-03, -9.03320312e-03,  1.07910156e-01, -1.74804688e-01,
         7.71484375e-02,  3.83377075e-04, -1.02539062e-01, -1.73339844e-02,
        -3.08837891e-02,  5.76171875e-02, -1.09863281e-01,  6.10351562e-02,
         2.48046875e-01,  5.46264648e-03,  3.49121094e-02,  7.65991211e-03,
        -1.07910156e-01,  2.16796875e-01,  1.26953125e-01,  1.46484375e-01,
         1.55273438e-01,  4.46777344e-02,  7.51953125e-02, -1.45507812e-01,
        -7.71484375e-02, -8.54492188e-02, -1.15966797e-02,  7.95898438e-02,
        -1.94335938e-01, -2.57812500e-01, -9.86328125e-02, -1.38671875e-01,
        -4.13894653e-04, -9.03320312e-02,  7.66601562e-02,  1.33789062e-01,
         5.17578125e-02, -4.80957031e-02,  1.88476562e-01, -3.44238281e-02,
        -7.95898438e-02,  3.54003906e-02, -1.02539062e-01, -1.33789062e-01,
        -1.06445312e-01,  7.42187500e-02, -2.46582031e-02,  1.99218750e-01,
         1.5

In [48]:
x.shape

AttributeError: 'list' object has no attribute 'shape'

In [50]:
x = np.reshape(x,[4,1200,1])
print(x)

[[[-0.02636719]
  [ 0.06835938]
  [-0.03112793]
  ...
  [-0.00616455]
  [-0.02478027]
  [ 0.10009766]]

 [[-0.06396484]
  [-0.22070312]
  [-0.20214844]
  ...
  [-0.16796875]
  [-0.02087402]
  [-0.14257812]]

 [[-0.06396484]
  [ 0.06835938]
  [ 0.22460938]
  ...
  [-0.03759766]
  [-0.05224609]
  [-0.03955078]]

 [[-0.01806641]
  [ 0.20605469]
  [ 0.23046875]
  ...
  [-0.296875  ]
  [ 0.125     ]
  [-0.00558472]]]


In [51]:
y = np.array([1,0,0,1],dtype='f')

In [52]:
y

array([1., 0., 0., 1.], dtype=float32)

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1197, 32)          160       
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 53,461
Trainable params: 53,461
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.fit(x,y,epochs=150,batch_size=4)

Epoch 1/150
4/4 [==============================] - 4s 916ms/step - loss: 7.9712 - acc: 0.5000
Epoch 2/150
4/4 [==============================] - 1s 135ms/step - loss: 7.9712 - acc: 0.5000
Epoch 3/150
4/4 [==============================] - 0s 107ms/step - loss: 7.9712 - acc: 0.5000
Epoch 4/150
4/4 [==============================] - 0s 108ms/step - loss: 7.9712 - acc: 0.5000
Epoch 5/150
4/4 [==============================] - 0s 107ms/step - loss: 7.9712 - acc: 0.5000
Epoch 6/150
4/4 [==============================] - 0s 106ms/step - loss: 7.9712 - acc: 0.5000
Epoch 7/150
4/4 [==============================] - 0s 109ms/step - loss: 7.9712 - acc: 0.5000
Epoch 8/150
4/4 [==============================] - 0s 108ms/step - loss: 7.9712 - acc: 0.5000
Epoch 9/150
4/4 [==============================] - 0s 109ms/step - loss: 7.9712 - acc: 0.5000
Epoch 10/150
4/4 [==============================] - 0s 107ms/step - loss: 7.9712 - acc: 0.5000
Epoch 11/150
4/4 [==============================] - 0s 123m

Epoch 88/150
4/4 [==============================] - 0s 107ms/step - loss: 7.9712 - acc: 0.5000
Epoch 89/150
4/4 [==============================] - 0s 106ms/step - loss: 7.9712 - acc: 0.5000
Epoch 90/150
4/4 [==============================] - 0s 106ms/step - loss: 7.9712 - acc: 0.5000
Epoch 91/150
4/4 [==============================] - 0s 106ms/step - loss: 7.9712 - acc: 0.5000
Epoch 92/150
4/4 [==============================] - 0s 105ms/step - loss: 7.9712 - acc: 0.5000
Epoch 93/150
4/4 [==============================] - 0s 106ms/step - loss: 7.9712 - acc: 0.5000
Epoch 94/150
4/4 [==============================] - 0s 106ms/step - loss: 7.9712 - acc: 0.5000
Epoch 95/150
4/4 [==============================] - 0s 106ms/step - loss: 7.9712 - acc: 0.5000
Epoch 96/150
4/4 [==============================] - 0s 108ms/step - loss: 7.9712 - acc: 0.5000
Epoch 97/150
4/4 [==============================] - 0s 107ms/step - loss: 7.9712 - acc: 0.5000
Epoch 98/150
4/4 [==============================] 